clean data

In [174]:
import pandas as pd

In [175]:
df = pd.read_csv(r"C:\Users\felm2\Desktop\AIAJKAEW - Sheet1.csv",encoding='tis-620')

In [176]:
df = df.rename(columns={"สถานะ":"label"})
df


,Timestamp,Soil,Temp,humid,pressure,x,y,z,label
0,10/2/2025 18:17,116,27.06,56.83398,1005.42828,88,89,175,ปกติ
1,10/2/2025 18:18,139,25.78,56.46777,1005.44531,89,86,175,ปกติ
2,10/2/2025 18:19,130,25.74,56.55469,1005.42578,89,86,175,ปกติ
3,10/2/2025 18:19,146,25.73,56.95020,1005.43268,89,86,175,ปกติ
4,10/2/2025 18:19,142,25.74,57.44922,1005.45544,88,85,175,ปกติ
...,...,...,...,...,...,...,...,...,...
3757,10/3/2025 22:51,2227,27.25,80.43555,1012.01764,86,22,142,ดินทรุด
3758,10/3/2025 22:51,702,27.25,80.37988,1012.02374,66,46,162,ดินทรุด
3759,10/3/2025 22:51,657,27.24,80.29883,1012.02307,66,25,150,ดินทรุด
3760,10/3/2025 22:51,617,27.26,80.27930,1012.04163,41,56,142,ดินทรุด


In [177]:
has_zero = (df == 0).to_numpy().any()
has_zero

np.True_

In [178]:
zero_total = (df==0).sum().sum()
zero_total

np.int64(10)

In [179]:
col_with_zero = df.columns[(df==0).any()]
col_with_zero

Index(['Soil'], dtype='object')

In [180]:
df = df[(df!=0).all(axis=1)]

In [181]:
has_zero = (df == 0).to_numpy().any()
has_zero

np.False_

In [182]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report, f1_score

In [183]:
seed = 6956


In [ ]:
df = df.sort_values("Timestamp").reset_index(drop=True)
num_cols = [c for c in df.columns if c not in ["label","Timestamp"] and np.issubdtype(df[c].dtype, np.number)]

for c in num_cols:
    
    df[f"{c}_mean_5"] = df[c].rolling(5, min_periods=1).mean().shift(1)
    df[f"{c}_std_5"]  = df[c].rolling(5, min_periods=1).std().shift(1)
    
    df[f"{c}_mean_10"] = df[c].rolling(10, min_periods=1).mean().shift(1)
    df[f"{c}_std_10"]  = df[c].rolling(10, min_periods=1).std().shift(1)
    
    df[f"{c}_diff1"] = df[c].diff(1)

In [185]:
df = df.dropna().reset_index(drop=True)

In [186]:
label_order = ["ปกติ","เสี่ยงดินทรุด","ดินทรุด"]
df = df[df["label"].isin(label_order)]
y = df["label"].map({l:i for i,l in enumerate(label_order)}).astype(int)
y

x = df.drop(columns=["label","Timestamp"])

In [187]:
from sklearn.model_selection import GroupKFold
groups = pd.to_datetime(df["Timestamp"]).dt.floor("min")
gkf = GroupKFold(n_splits=5)

In [188]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

train 5 fold

In [ ]:
f1s = []

for fold,(tr,te) in enumerate(gkf.split(x,y,groups),1):
    x_tr, x_te = x.iloc[tr], x.iloc[te]
    y_tr, y_te = y.iloc[tr], y.iloc[te]

    
    cw = compute_class_weight("balanced", classes=np.unique(y_tr), y=y_tr)
    sw = y_tr.map(dict(zip(np.unique(y_tr), cw))).to_numpy()

    clf = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        n_estimators=2000,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        tree_method="hist",
        random_state=6956,
        eval_metric="mlogloss"
    )

    
    cutoff = int(len(x_tr)*0.9)
    clf.fit(
        x_tr.iloc[:cutoff], y_tr.iloc[:cutoff],
        sample_weight=sw[:cutoff],
        eval_set=[(x_tr.iloc[cutoff:], y_tr.iloc[cutoff:])],
        verbose=False
    )

    pred = clf.predict(x_te)
    f1 = f1_score(y_te, pred, average="macro")
    f1s.append(f1)
    print(f"[Fold {fold}] F1-macro = {f1:.4f}")

[Fold 1] F1-macro = 0.6119
[Fold 2] F1-macro = 1.0000
[Fold 3] F1-macro = 0.9334
[Fold 4] F1-macro = 1.0000
[Fold 5] F1-macro = 0.8930


In [190]:
print("CV F1-macro: %.4f ± %.4f" % (np.mean(f1s), np.std(f1s)))
print(classification_report(y_te, pred, target_names=label_order, digits=4))

CV F1-macro: 0.8877 ± 0.1438
               precision    recall  f1-score   support

         ปกติ     0.9932    1.0000    0.9966       727
เสี่ยงดินทรุด     1.0000    0.7895    0.8824        19
      ดินทรุด     1.0000    0.6667    0.8000         3

     accuracy                         0.9933       749
    macro avg     0.9977    0.8187    0.8930       749
 weighted avg     0.9934    0.9933    0.9929       749



train all

In [191]:
clf_final = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    tree_method="hist",
    random_state=6956,
    eval_metric="mlogloss"
)
clf_final.fit(x, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2000,
              n_jobs=None, num_class=3, num_parallel_tree=None, ...)

In [193]:
pred = clf.predict(x_te)
print(classification_report(y_te, pred, target_names=label_order, digits=4))

               precision    recall  f1-score   support

         ปกติ     0.9932    1.0000    0.9966       727
เสี่ยงดินทรุด     1.0000    0.7895    0.8824        19
      ดินทรุด     1.0000    0.6667    0.8000         3

     accuracy                         0.9933       749
    macro avg     0.9977    0.8187    0.8930       749
 weighted avg     0.9934    0.9933    0.9929       749



test real data

In [256]:
new = pd.read_csv(r"C:\Users\felm2\Desktop\new_test.csv", encoding="tis-620")
new = new.sort_values("Timestamp").reset_index(drop=True)

num_cols = [c for c in new.columns if c not in ["label","Timestamp"] and np.issubdtype(new[c].dtype, np.number)]
for c in num_cols:
    new[f"{c}_mean_5"]  = new[c].rolling(5, min_periods=1).mean().shift(1)
    new[f"{c}_std_5"]   = new[c].rolling(5, min_periods=1).std().shift(1)
    new[f"{c}_mean_10"] = new[c].rolling(10, min_periods=1).mean().shift(1)
    new[f"{c}_std_10"]  = new[c].rolling(10, min_periods=1).std().shift(1)
    new[f"{c}_diff1"]   = new[c].diff(1)
new = new.dropna().reset_index(drop=True)

x_real = new.drop(columns=["Timestamp","label"], errors="ignore")
y_true=new["label"]
y_true = new["label"].map({l:i for i,l in enumerate(label_order)}).astype(int)
x_real




,Soil,Temp,humid,pressure,x,y,z,Soil_mean_5,Soil_std_5,Soil_mean_10,...,y_mean_5,y_std_5,y_mean_10,y_std_10,y_diff1,z_mean_5,z_std_5,z_mean_10,z_std_10,z_diff1
0,1908,27.40,78.56152,1011.90094,95,89,175,1463.500000,195.868578,1463.500000,...,88.000000,0.000000,88.000000,0.000000,1.0,175.0,0.0,175.0,0.0,0.0
1,1976,27.39,78.57129,1011.89087,96,89,175,1611.666667,291.620187,1611.666667,...,88.333333,0.577350,88.333333,0.577350,0.0,175.0,0.0,175.0,0.0,0.0
2,2060,27.39,78.55957,1011.88678,95,89,175,1702.750000,299.799238,1702.750000,...,88.500000,0.577350,88.500000,0.577350,0.0,175.0,0.0,175.0,0.0,0.0
3,2095,27.38,78.55859,1011.91016,96,89,175,1774.200000,304.852751,1774.200000,...,88.600000,0.547723,88.600000,0.547723,0.0,175.0,0.0,175.0,0.0,0.0
4,2139,27.38,78.58105,1011.90436,96,89,175,1928.200000,196.400611,1827.666667,...,88.800000,0.447214,88.666667,0.516398,0.0,175.0,0.0,175.0,0.0,0.0


predict

In [258]:
y_pred = clf_final.predict(x_real)
y_prob = clf_final.predict_proba(x_real)


In [264]:
y_pred


array([0, 0, 1, 1, 1])

In [265]:
y_true

0    0
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [271]:
%pip install onnxmltools onnxruntime


Note: you may need to restart the kernel to use updated packages.


In [274]:
# 1) ทำ mapping ชื่อคอลัมน์ -> f0,f1,...
cols = list(x.columns)
name_map = {c: f"f{i}" for i,c in enumerate(cols)}

# 2) ตั้งชื่อฟีเจอร์ให้ booster เป็น f0..f{n-1}
booster = clf_final.get_booster()
booster.feature_names = [name_map[c] for c in cols]
booster.feature_types = None  # กัน warning เรื่องชนิดข้อมูล

# 3) แปลงเป็น ONNX ด้วย onnxmltools
import onnxmltools
from onnxmltools.convert.common.data_types import FloatTensorType

initial_type = [("float_input", FloatTensorType([None, x.shape[1]]))]
onnx_model = onnxmltools.convert_xgboost(booster, initial_types=initial_type, target_opset=13)

with open("model.onnx","wb") as f:
    f.write(onnx_model.SerializeToString())

# 4) บันทึก mapping ไว้ใช้ตอน inference ฝั่งเว็บ
import json
with open("feature_mapping.json","w",encoding="utf-8") as f:
    json.dump({"order": cols, "onnx_names": [name_map[c] for c in cols]}, f, ensure_ascii=False, indent=2)
